In [ ]:
import json
import random
import subprocess
from datetime import datetime, timedelta, timezone

# ==== CONFIG ====
NUM_PATIENTS = 1
NUM_DAYS = 1


In [ ]:


# ==== MODEL CALLER ====
def call_ollama(model, prompt):
    print(f"\n[DEBUG] Calling model: {model}")
    print(f"[DEBUG] Prompt:\n{prompt}")
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode("utf-8"),
        stdout=subprocess.PIPE
    )
    output = result.stdout.decode("utf-8").strip()
    print(f"[DEBUG] Raw Model Output:\n{output}")
    return output

# ==== GENERATION FUNCTIONS ====
def generate_vitals():
    vitals = {
        "heart_rate": {"value": random.randint(60, 100), "unit": "bpm"},
        "spo2": {"value": random.randint(92, 99), "unit": "%"},
        "temperature": {"value": round(random.uniform(36.0, 38.0), 1), "unit": "°C"},
        "bp": {
            "systolic": random.randint(110, 160),
            "diastolic": random.randint(70, 100),
            "unit": "mmHg"
        }
    }
    print(f"[DEBUG] Generated Vitals: {vitals}")
    return vitals

def generate_symptoms(vitals):
    prompt = f"Patient vitals: {json.dumps(vitals)}"
    try:
        symptoms = json.loads(call_ollama("symptom_generator", prompt))
        if not symptoms:
            raise ValueError("Empty symptoms list")
        return symptoms
    except:
        print("[WARNING] symptom_generator failed, using fallback symptoms.")
        return ["fatigue"]

def generate_medications(symptoms):
    prompt = f"Patient symptoms: {json.dumps(symptoms)}"
    try:
        meds = json.loads(call_ollama("medication_planner", prompt))
        if not meds:
            raise ValueError("Empty medications list")
        return meds
    except:
        print("[WARNING] medication_planner failed, using fallback meds.")
        return [{"name": "Paracetamol", "dose": "500mg", "scheduled_time": "08:00", "compliance": "Taken"}]

def generate_nursing_note(symptoms, vitals, medications):
    prompt = f"""
    Symptoms: {symptoms}
    Vitals: {vitals}
    Medications: {medications}
    """
    return call_ollama("nursing_note_generator", prompt)

def generate_clinical_summary(nursing_note):
    prompt = f"Summarise this nursing note into 3–5 lines:\n{nursing_note}"
    return call_ollama("guardian_nurse", prompt)

# ==== MAIN RECORD GENERATOR ====
def generate_patient_record(patient_id, date):
    vitals = generate_vitals()
    symptoms = generate_symptoms(vitals)
    medications = generate_medications(symptoms)
    nursing_note = generate_nursing_note(symptoms, vitals, medications)
    clinical_summary = generate_clinical_summary(nursing_note)

    record = {
        "patient_id": f"P{patient_id:05d}",
        "age": random.randint(65, 95),
        "gender": random.choice(["Male", "Female"]),
        "timestamp": date.isoformat(),
        "nursing_note": nursing_note,
        "medications": medications,
        "vitals": vitals,
        "adls": {
            "steps_taken": random.randint(1000, 5000),
            "calorie_intake": random.randint(800, 2000),
            "sleep_hours": random.randint(4, 9)
        },
        "behaviour_tags": [],
        "emotion_analysis": {"tags": [], "confidence_scores": {}},
        "clinical_summary": clinical_summary,
        "entities_extracted": {},
        "baseline_stats": {
            "avg_heart_rate": None,
            "avg_spo2": None,
            "avg_sleep_hours": None,
            "avg_calorie_intake": None,
            "usual_diet_compliance": None
        },
        "alerts": []
    }
    print(f"\n[DEBUG] Final Patient Record:\n{json.dumps(record, indent=2)}\n")
    return record

# ==== DATASET GENERATION ====
def main():
    start_date = datetime.now(timezone.utc) - timedelta(days=NUM_DAYS)
    for pid in range(1, NUM_PATIENTS + 1):
        for d in range(NUM_DAYS):
            date = start_date + timedelta(days=d)
            generate_patient_record(pid, date)

if __name__ == "__main__":
    main()
